# Notebook for text extraction on image
Inga Ulusoy, SSC, July 2022

In [ ]:
import os
from IPython.display import Image, display
import misinformation

In [ ]:
images = misinformation.find_files(path="../data/images-little-text/", limit=1000)

In [ ]:
for i in images[0:10]:
    display(Image(filename=i))

In [ ]:
# start with only English
mysubfiles = [i for i in images if "eng" in i]

In [ ]:
for i in mysubfiles[0:10]:
    display(Image(filename=i))

In [ ]:
mydict = misinformation.utils.initialize_dict(mysubfiles[0:10])

In [ ]:
mydict

# google cloud vision API
First 1000 images per month are free.

In [ ]:
os.environ[
    "GOOGLE_APPLICATION_CREDENTIALS"
] = "../data/seismic-bonfire-329406-412821a70264.json"

## Inspect the elements per image

In [ ]:
# misinformation.explore_analysis(mydict, identify="text-on-image")

## Or directly analyze for further processing

In [ ]:
for key in mydict:
    print(key)
    mydict[key] = misinformation.text.TextDetector(mydict[key]).analyse_image()

In [ ]:
print(mydict["104157S_eng"]["text"])

In [ ]:
test = mydict["104157S_eng"]["text"][0]
print(test)

In [ ]:
# flake8-noqa-cell
from googletrans import Translator

In [ ]:
translator = Translator()

In [ ]:
result = translator.translate(test)

In [ ]:
print(result.text)
print(result.src)
print(result.origin)

In [ ]:
# now clean the gibberish
# do spell check
# flake8-noqa-cell
import contextualSpellCheck
import spacy

nlp = spacy.load("en_core_web_md")
contextualSpellCheck.add_to_pipe(nlp)

In [ ]:
doc = nlp(result.text)
print(doc._.outcome_spellCheck)

In [ ]:
final_text = []
for token in doc:
    if token.pos_ != "SPACE":
        if token.pos_ != "NUM":
            if token.has_vector:
                final_text.append(token.text)

In [ ]:
print(" ".join(final_text))